# Imports

In [1]:
import numpy as np
import cupy as cp
from numpy import format_float_scientific as fs
import matplotlib.pyplot as plt
from matplotlib import cm
import time

from lbmFlowAroundCylinder import Timer, TimersManager
from lbmFlowAroundCylinder import inivel, obstacle_fun

## Timers definition

In [2]:
timers = TimersManager()
timers.add("main")
timers.add("equilibrium")
timers.add("collision")
timers.add("streaming")
timers.add("macroscopic")
timers.add("rightwall")
timers.add("leftwall")
timers.add("fineq")
timers.add("bounceback")
timers.add("move_gpu->cpu")
timers.add("move_cpu->gpu")

## Flow definitions

In [3]:
maxIter = 2000    # Total number of time iterations.
Re = 150.0          # Reynolds number.
nx, ny = 420, 180   # Numer of lattice nodes.
ly = ny-1           # Height of the domain in lattice units.
cx, cy, r = nx//4, ny//2, ny//9 # Coordinates of the cylinder.
uLB     = 0.04                  # Velocity in lattice units.
nulb    = uLB*r/Re;             # Viscoscity in lattice units.
omega = 1 / (3*nulb+0.5);    # Relaxation parameter.
save_figures = False
profile = True

## Lattice constants

In [4]:
v = cp.array([ [ 1,  1], [ 1,  0], [ 1, -1], [ 0,  1], [ 0,  0],
               [ 0, -1], [-1,  1], [-1,  0], [-1, -1] ]) # 9 vecteurs : 9 directions de déplacement
v_np = cp.asnumpy(v)
t = cp.array([ 1/36, 1/9, 1/36, 1/9, 4/9, 1/9, 1/36, 1/9, 1/36])
t_np = cp.asnumpy(t)

col1 = np.array([0, 1, 2])
col2 = np.array([3, 4, 5])
col3 = np.array([6, 7, 8])

# Code main functions

### Macroscopic

In [5]:
def macroscopic(fin): 
    """Compute macroscopic variables (density, velocity)

    fluid density is 0th moment of distribution functions 
    fluid velocity components are 1st order moments of dist. functions
    """
    timers.get("macroscopic").start()
    rho = cp.sum(fin, axis=0)
    u = cp.zeros((2, nx, ny))
    for i in range(9):
        u[0,:,:] += v[i,0] * fin[i,:,:]
        u[1,:,:] += v[i,1] * fin[i,:,:]
    u /= rho
    timers.get("macroscopic").end()
    return rho, u

### Equilibrium

In [6]:
def equilibrium(rho, u):
    """Equilibrium distribution function.
    """
    timers.get("equilibrium").start()
    usqr = 3/2 * (u[0]**2 + u[1]**2)
    feq = cp.zeros((9,nx,ny))
    for i in range(9):
        cu = 3 * (v[i,0]*u[0,:,:] + v[i,1]*u[1,:,:])
        feq[i,:,:] = rho*t[i] * (1 + cu + 0.5*cu**2 - usqr) 
        # feq[i,:,:] : dimension 1 la direction de déplacement de la particule
        #               dimension 2 et 3 : x et y la position
    timers.get("equilibrium").end()
    return feq

### Main loop

In [7]:
def main(v, t):
    # create obstacle mask array from element-wise function
    obstacle = np.fromfunction(obstacle_fun, (nx,ny))
    obstacle = cp.asarray(obstacle)
    
    # initial velocity field vx,vy from element-wise function
    # vel is also used for inflow border condition
    vel = np.fromfunction(inivel, (2,nx,ny)) 
    vel = cp.asarray(vel)
    
    # Initialization of the populations at equilibrium 
    # with the given velocity.
    fin = equilibrium(1, vel) 
    timers.get("move_gpu->cpu").start()
    fin_np = cp.asnumpy(fin)
    timers.get("move_gpu->cpu").end()

    ###### Main time loop ########
    for time in range(maxIter):
        # Right wall: outflow condition.
        # we only need here to specify distrib. function for velocities
        # that enter the domain (other that go out, are set by the streaming step)
        
        timers.get("rightwall").start()
        fin_np[col3,nx-1,:] = fin_np[col3,nx-2,:] 
        timers.get("rightwall").end()

        timers.get("move_cpu->gpu").start()
        fin = cp.asarray(fin_np)
        timers.get("move_cpu->gpu").end()
        
        # Compute macroscopic variables, density and velocity.
        rho, u = macroscopic(fin) # Timer in func

        # Left wall: inflow condition.
        timers.get("leftwall").start()
        u[:,0,:] = vel[:,0,:]
        rho[0,:] = 1/(1-u[0,0,:]) * ( np.sum(fin[col2,0,:], axis=0) +
                                      2*np.sum(fin[col3,0,:], axis=0) )
        timers.get("leftwall").end()
        
        # Compute equilibrium.
        feq = equilibrium(rho, u) # Timer in func
        timers.get("fineq").start()
        fin[[0,1,2],0,:] = feq[[0,1,2],0,:] + fin[[8,7,6],0,:] - feq[[8,7,6],0,:]
        timers.get("fineq").end()

        # Collision step.
        timers.get("collision").start()
        fout = fin - omega * (fin - feq) # Noyau de calcul 1
        timers.get("collision").end()

        # Bounce-back condition for obstacle.
        # in python language, we "slice" fout by obstacle
        timers.get("bounceback").start()
        for i in range(9):
            fout[i, obstacle] = fin[8-i, obstacle]
        timers.get("bounceback").end()

        # Streaming step.
        
        timers.get("move_gpu->cpu").start()
        fin_np = cp.asnumpy(fin)
        timers.get("move_gpu->cpu").end()
        timers.get("move_gpu->cpu").start()
        fout_np = cp.asnumpy(fout)
        timers.get("move_gpu->cpu").end()
        
        timers.get("streaming").start()
        for i in range(9):
            fin_np[i,:,:] = np.roll(np.roll(fout_np[i,:,:], v_np[i,0], axis=0),
                                 v_np[i,1], axis=1 ) # Noyau de calcul 2
        timers.get("streaming").end()
        


In [8]:
timers.get("main").start()
main(v, t)
timers.get("main").end()

In [9]:
total = np.sum(timers.get("main").getMeasures())
print(f"Total time : {total:4.2f}s")
timers.printInfo()

Total time : 12.89s
--> Timer 'main         ' : N =    1 | Mean 1.289e+01 +- 0.e+00     | 100.0% of total time.
--> Timer 'equilibrium  ' : N = 2001 | Mean 1.331e-03 +- 2.162e-03  | 20.66% of total time.
--> Timer 'collision    ' : N = 2000 | Mean 4.18e-05  +- 9.578e-06  |  0.65% of total time.
--> Timer 'streaming    ' : N = 2000 | Mean 1.145e-03 +- 1.178e-05  | 17.76% of total time.
--> Timer 'macroscopic  ' : N = 2000 | Mean 5.179e-04 +- 3.442e-05  |  8.03% of total time.
--> Timer 'rightwall    ' : N = 2000 | Mean 1.522e-05 +- 2.059e-06  |  0.24% of total time.
--> Timer 'leftwall     ' : N = 2000 | Mean 3.789e-04 +- 8.637e-05  |  5.88% of total time.
--> Timer 'fineq        ' : N = 2000 | Mean 3.635e-04 +- 1.468e-05  |  5.64% of total time.
--> Timer 'bounceback   ' : N = 2000 | Mean 1.098e-03 +- 6.084e-05  | 17.03% of total time.
--> Timer 'move_gpu->cpu' : N = 4001 | Mean 5.038e-04 +- 7.046e-05  | 15.63% of total time.
--> Timer 'move_cpu->gpu' : N = 2000 | Mean 5.106e-04 +- 6.4